TextBlobでの感情分析に関するテストコード  
Driverless AI Recipe（[sentiment_score.py](https://github.com/h2oai/driverlessai-recipes/blob/dai-1.9/data/nlp/sentiment_score.py)）カスタマイズの参考

TextBlob Document:  
[Official](https://textblob.readthedocs.io/en/dev/#)  
[Official - API](https://textblob.readthedocs.io/en/dev/api_reference.html)  
  
Blog  
[【python】テキスト処理に使えるTextBlobを使う](https://qiita.com/yasutaka_ono/items/6a565c333995d43918ff)

In [3]:
import pandas as pd
from textblob import TextBlob

In [4]:
df = pd.read_csv("test_data/T15-2020-1-7-SAMPLE.csv")
df.shape

(30, 5)

In [5]:
df.head()

,ID,JP_Original,JP_Simple,EN_Original,Negative
0,1,誰が一番に着くか私には分かりません。,誰が一番に着くか私には分かりません。,i can 't tell who will arrive first .,1
1,2,多くの動物が人間によって滅ぼされた。,多くの動物が人間によって殺された。,many animals have been destroyed by men .,1
2,3,私はテニス部員です。,私はテニス部員です。,i 'm in the tennis club .,0
3,4,エミは幸せそうに見えます。,エミは幸せそうに見えます。,emi looks happy .,0
4,5,この事実を心に留めておいて下さい。,この事実を覚えておいてください。,please bear this fact in mind .,0


In [8]:
sample_jp = df["JP_Original"][0]
sample_jp

'誰が一番に着くか私には分かりません。'

In [9]:
sample_en = df["EN_Original"][0]
sample_en

"i can 't tell who will arrive first ."

In [10]:
# translate to English
TextBlob(sample_jp).translate(to='en')

TextBlob("I don't know who will arrive first.")

In [11]:
# translate to Japanese
TextBlob(sample_en).translate(to='ja')

TextBlob("誰が最初に到着するのかわかりません。")

In [12]:
TextBlob("Hello my friend.").detect_language()

'en'

In [13]:
TextBlob("彼はFriendのことがLoveだ。").detect_language()

'ja'

In [14]:
TextBlob("彼はFriendのことがLoveだ。But not in the past.").detect_language()

'ja'

In [15]:
TextBlob("彼はFriendのことがLoveだ。But not in the past.").translate(to='en')

TextBlob("He loves Friend. But not in the past.")

In [16]:
TextBlob("Not in the past. But,彼はFriendのことがLoveだ。").detect_language()

'ja'

In [17]:
TextBlob("Not in the past. But,彼はFriendのことがLoveだ。").translate(to='en')

TextBlob("Not in the past. But, he loves Friend.")

In [18]:
# 単語のリスト化
TextBlob(sample_en).words

WordList(['i', 'can', "'t", 'tell', 'who', 'will', 'arrive', 'first'])

In [19]:
# 日本語はできない
TextBlob(sample_jp).words

WordList(['誰が一番に着くか私には分かりません。'])

In [20]:
# sentiment analysis
TextBlob(sample_en).sentiment

Sentiment(polarity=0.25, subjectivity=0.3333333333333333)

In [21]:
TextBlob(sample_en).sentiment[0]

0.25

In [22]:
# sentiment analysis -> 日本語はできない
TextBlob(sample_jp).sentiment

Sentiment(polarity=0.0, subjectivity=0.0)

In [23]:
# sentiment analysis -> 日本語は翻訳してから実施
TextBlob(sample_jp).translate(to='en').sentiment

Sentiment(polarity=0.25, subjectivity=0.3333333333333333)

#### en, ja両方に対応できる関数の定義

In [25]:
def sentiment_score(x):
            """
            x: text(str type) line
            return: polarity
            """
            if TextBlob(x).detect_language()=="en":
                return TextBlob(x).sentiment[0]
            elif TextBlob(x).detect_language()=="ja":
                return TextBlob(x).translate(to='en').sentiment[0]
            else:
                return 0

In [29]:
print(sample_jp)
sentiment_score(sample_jp)

誰が一番に着くか私には分かりません。


0.25

In [30]:
print(sample_en)
sentiment_score(sample_en)

i can 't tell who will arrive first .


0.25

#### 列への適用

In [31]:
df["JP_Original"].astype(str).fillna("NA").apply(sentiment_score)

0     0.25
1     0.50
2     0.00
3     0.80
4     0.00
5     0.00
6     0.00
7     0.00
8     0.00
9    -0.15
10    0.00
11    0.00
12    0.00
13    0.00
14    0.00
15    0.00
16    0.00
17    0.00
18    0.00
19    0.35
20    0.00
21    0.55
22    0.00
23    0.00
24    0.00
25    0.00
26    0.00
27   -0.05
28    0.00
29    0.00
Name: JP_Original, dtype: float64

In [32]:
df["EN_Original"].astype(str).fillna("NA").apply(sentiment_score)

0     0.250000
1     0.500000
2     0.000000
3     0.000000
4     0.000000
5     0.000000
6     0.000000
7     0.000000
8     0.000000
9     0.136364
10    0.000000
11    0.000000
12    0.000000
13    0.000000
14    0.000000
15    0.000000
16    0.000000
17    0.000000
18    0.000000
19    0.350000
20   -0.166667
21    0.550000
22   -0.200000
23   -0.050000
24    0.000000
25    0.000000
26    0.000000
27   -0.050000
28    0.000000
29    0.000000
Name: EN_Original, dtype: float64